# Looker Data Dictionary 

Before running the script, follow the instructions at https://github.com/llooker/python_api_samples to configure a config file and save it in your working directory

Import packages

In [95]:
import lookerapi
import pandas as pd
import warnings
import numpy as np
from google.oauth2 import service_account
import pandas_gbq
import os
#warnings.filterwarnings("ignore")

Connect to your Looker Instance

In [96]:
#assumes that you the config.yml in current directory
config_filename = 'saleseng_config.yml'
looker = lookerapi.connect_looker(config_filename)

Enter a model name you want to create the data dictionary on

In [97]:
model_name = 'thelook_leigha'
model = looker.get_model(model_name) 

https://saleseng.dev.looker.com:19999/api/3.1/lookml_models/thelook_leigha


Iterate through all the explores and create a dataframe with the metadata for each field

In [105]:
field_dict = []
for explore_n in model['explores']:
    explore = looker.get_explore(explore_name=explore_n["name"],model_name=model_name)
    dimensions = explore['fields']['dimensions']
    for d in dimensions:
        d['explore'] = explore['label']
        d['explore_name'] = explore['name']
        d['model_name'] = model_name
        #d['explore_link'] = looker.host.replace(':19999/api/3.1/','')+'/explore/'+model_name+'/'+explore['name']
        d['explore_description'] = explore['description']
    measures = explore['fields']['measures']
    for m in measures:
        m['explore'] = explore['label']
        m['explore_name'] = explore['name']
        m['model_name'] = model_name
        #m['explore_link'] = looker.host.replace(':19999/api/3.1/','')+'/explore/'+model_name+'/'+explore['name']
        m['explore_description'] = explore['description']
    field_dict = field_dict+dimensions+measures

In [106]:
field_df = pd.DataFrame(field_dict)

Write metadata results back to database

In [107]:
credentials = service_account.Credentials.from_service_account_file(os.environ['GOOGLE_APPLICATION_CREDENTIALS'])

In [108]:
dataset='exoplanets'
project='looker-se'
table_name = 'looker_data_dictionary'
pandas_gbq.to_gbq(field_df.reset_index(), dataset+'.'+table_name, project, if_exists='replace')

1it [00:05,  5.24s/it]
